In [227]:
import pandas as pd
import numpy as np

### Importing Data

In [228]:
data=pd.read_csv('marriage.csv')

### EDA

In [229]:
data.head()

,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


In [230]:
data.shape

(2567, 10)

In [231]:
(data.isnull().sum()/data.shape[0])*100

id                  0.000000
gender              1.129723
height              4.596806
religion           24.737047
caste               5.531749
mother_tongue       6.388781
profession         12.855473
location            6.038177
country             0.623296
age_of_marriage     0.740164
dtype: float64

In [232]:
#droping nan values

In [233]:
data.dropna(inplace=True)

In [234]:
data.shape

(1932, 10)

In [235]:
#encoding

In [236]:
x=data.drop('age_of_marriage',axis=1)
y=data['age_of_marriage']

In [237]:
#height is not categorical

In [238]:
len(x['gender'].value_counts())

2

In [239]:
len(x['religion'].value_counts())

5

In [240]:
len(x['caste'].value_counts())

41

In [241]:
len(x['mother_tongue'].value_counts())

27

In [242]:
len(x['profession'].value_counts())

84

In [243]:
len(x['location'].value_counts())

461

In [244]:
len(x['country'].value_counts())

22

In [245]:
x.drop(['id','profession'],axis=1,inplace=True)

In [246]:
x=x.reset_index()

In [247]:
#converting height into cm
#((foot×12)+inch) x 2.54 

In [248]:
height_cm=[]
for i in range(len(x)):
    a=float(x.loc[i,'height'].split('\'')[0])
    b=float(x.loc[i,'height'].split('\'')[1].replace('"',''))
    cms=((a*12)+b)*2.54
    height_cm.append(round(cms,2))

In [249]:
x['height_cm']=height_cm

In [250]:
x.drop('height',axis=1,inplace=True)

In [251]:
x.drop('index',axis=1,inplace=True)

In [252]:
x.head()

,gender,religion,caste,mother_tongue,location,country,height_cm
0,male,Jain,Shwetamber,Gujarati,Fairfax- VA,USA,170.18
1,male,Hindu,Brahmin,Hindi,Begusarai,India,170.18
2,female,Hindu,Thakur,Hindi,Mumbai,India,152.40
3,male,Christian,Born Again,Malayalam,Sulthan Bathery,India,165.10
4,male,Hindu,Valmiki,Hindi,Delhi,India,165.10


In [253]:
y=y.reset_index()

In [254]:
y=y.drop('index',axis=1)

### Encoding

In [255]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
x=x.apply(l.fit_transform)

In [256]:
x

,gender,religion,caste,mother_tongue,location,country,height_cm
0,1,2,34,6,153,19,14
1,1,1,14,8,56,5,14
2,0,1,36,8,295,5,7
3,1,0,13,13,410,5,12
4,1,1,38,8,124,5,12
...,...,...,...,...,...,...,...
1927,1,1,21,18,295,5,18
1928,0,1,36,8,5,5,10
1929,1,1,36,8,325,3,18
1930,0,1,7,2,60,5,10


##### building model

In [284]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [285]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=80,max_depth=20)
rf.fit(x_train,y_train)

C:\Users\HP\AppData\Local\Temp\ipykernel_19004\2985290293.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train,y_train)


RandomForestRegressor(max_depth=20, n_estimators=80)

In [286]:
ypred=rf.predict(x_test)

In [287]:
from sklearn.metrics import r2_score
r2_score(y_test,ypred)

0.7045403146305327

In [288]:
pred2=rf.predict(x_train)

In [289]:
r2_score(y_train,pred2)

0.9388378479101241

In [290]:
# from sklearn.model_selection import RandomizedSearchCV

# n_estimators=[int(i) for i in np.linspace(100,1000,num=100)]
# criterion=['squared_error', 'absolute_error', 'poisson']
# max_features=['sqrt', 'log2', None]
# n_jobs=[-1,1]

# param_grid={'n_estimators':n_estimators,
#            'criterion':criterion,
#            'max_features':max_features,
#            'n_jobs':n_jobs}

# rf=RandomForestRegressor()
# rCV=RandomizedSearchCV(estimator=rf,param_distributions=param_grid,scoring='neg_mean_squared_error',n_iter=10,cv=3,random_state=42,n_jobs=1)

In [291]:
# rCV.fit(x_train,y_train)

In [292]:
# p=rCV.predict(x_test)

In [293]:
# r2_score(y_test,p)

In [294]:
# pred1=rCV.predict(x_train)

In [295]:
# r2_score(y_train,pred1)

In [296]:
import pickle
model=pickle.dump(rf,open('rf_model.pkl','wb'))

In [297]:
data=pd.concat([x,y],axis=1)

In [298]:
data.to_csv('marriage_data.csv')